In [5]:
import os
import sys
import numpy as np
import pandas as pd
import requests
from tqdm.notebook import tqdm as tqdm
from functools import partial
from pathlib import Path
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool 
import errno
import time
import pysftp

In [6]:
# HELPERS
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python > 2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
        
# HELPERS

In [7]:
with open('/Users/quanle/locca_revolve/input/test_urls.txt', 'r') as f:
    test_urls = f.readlines()
test_urls = [x.strip() for x in test_urls]
in_urls = test_urls

In [8]:
with open('/Users/quanle/locca_revolve/input/user_agents.txt', 'r') as f:
    user_agents = f.readlines()
user_agents = [x.strip() for x in user_agents]

In [9]:
url_prefix = 'https://chroniclingamerica.loc.gov/data/batches/'
batch_names = [x.replace(url_prefix, '').split('/')[0] for x in test_urls]
url_tails = [x.replace(url_prefix, '').split('/')[1:] for x in test_urls]
url_tails = ['/'.join(x) for x in url_tails]

out_path = "./scans/"
out_files = [x.replace(url_prefix, out_path) for x in test_urls]

In [10]:
in_zip = list(zip(in_urls, out_files))

In [11]:
my_chunks = list(chunks(in_zip, 5))

In [12]:
def parallelize_download(chunk):
    for i in range(len(chunk)):
        HEADERS = {
            'User-Agent': np.random.choice(user_agents)
        }
        out_folder = '/'.join(chunk[i][1].split('/')[:-1])
        mkdir_p(out_folder)
        r = requests.get(chunk[i][0], headers=HEADERS)
        with open(in_zip[i][1], 'wb') as f:
            f.write(r.content)
    return 0          

In [13]:
pool = ThreadPool(32)
time0 = time.time()
results = list(tqdm(pool.imap(parallelize_download, my_chunks)))
time1 = time.time()
print('Time elapsed: ', time1-time0)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [14]:
log_file = './log_%s.txt' % time.strftime("%Y%m%d-%H%M%S")

In [24]:
with open("secret_pw.txt", 'r') as f:
    secret_pw = f.read().strip()

with pysftp.Connection('malloy.princeton.edu', username='qle', password=secret_pw) as sftp:
    for i in range(len(in_zip)):
        with sftp.cd('locca_revolve/'):
            out_folder = '/'.join(in_zip[i][1].split('/')[:-1])
            mkdir_p(out_folder)
            mkdir_p('/'.join(['scans', batch_names[i]]))
            sftp.get(in_zip[i][1], in_zip[i][1])

In [26]:
in_zip

[('https://chroniclingamerica.loc.gov/data/batches/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0007.jp2',
  './scans/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0007.jp2'),
 ('https://chroniclingamerica.loc.gov/data/batches/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0008.jp2',
  './scans/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0008.jp2'),
 ('https://chroniclingamerica.loc.gov/data/batches/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0009.jp2',
  './scans/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0009.jp2'),
 ('https://chroniclingamerica.loc.gov/data/batches/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0010.jp2',
  './scans/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0010.jp2'),
 ('https://chroniclingamerica.loc.gov/data/batches/ak_bowheadwhale_ver01/data/sn83045499/00514159531/1929010101/0011.jp2',
  './scans/ak_bowheadwhale_ver01/data/sn83045499/